<a href="https://colab.research.google.com/github/NeginZarbakhsh/Mapillary-Points-of-Interest/blob/main/Codes/1_Retrieve_all_Mapillary_images_within_the_radius_of_latitude_and_longitude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs the package. Google Colab may ask you to restart the runtime
!pip install mapillary

In [ ]:
# Import all the required libraries
import requests
import pandas as pd
import json
import mapillary.interface as mly
from mapillary.models.geojson import GeoJSON
from mapillary.models.client import Client
from mapillary.config.api.entities import Entities

# Mapillary API
**Set the token for Mapillary API**:

1.   Open [Mapillary website](https://www.mapillary.com/dashboard/developer),
2.   Go to 'developers',
3.  'Register application', register a new application,
4.   Copy & paste the 'Client Token' in access_token 



In [ ]:
access_token = 'MLY|YYY'
mly.set_access_token(access_token)

{'token': 'SUCCESS'}

# Safegraph Sample Data

**Request data from Safegraph via their website**: We downloaded the sample data from their website to demonstrate how to use Safegraph in conjunction with Mapillary; please see the [Safegraph website](https://https://www.safegraph.com/free-data/global-brands-sample-data) for more information on data collection.

In [ ]:
safegraph_place = pd.read_excel('/content/SafeGraph-Global-Places-Sample-Q3-2022.xlsx')
#Choose the fist 5 items in sample data to test the code
safegraph_place = safegraph_place.head(5)
safegraph_place

,PLACEKEY,PARENT_PLACEKEY,SAFEGRAPH_BRAND_IDS,LOCATION_NAME,BRANDS,STORE_ID,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,...,REGION,ISO_COUNTRY_CODE,POSTAL_CODE,OPEN_HOURS,CATEGORY_TAGS,OPENED_ON,CLOSED_ON,TRACKING_CLOSED_SINCE,WEBSITES,PHONE_NUMBER
0,zzy-224@cdn-8q6-vfz,NaN,SG_BRAND_75e52dcc790fbad91ae83227c0fb6e2f,KFC,KFC,135,Restaurants and Other Eating Places,Limited-Service Restaurants,722513,1.385253,...,North West,SG,671524,"{\n ""Fri"": [\n [\n ""11:00"",\n ""2...","Counter Service,Lunch,Fast Food,Drive Through,...",NaT,NaT,2021-08-01,"[\n ""kfc.com.sg""\n]",6.567690e+09
1,zzy-222@7ds-5nh-t7q,NaN,SG_BRAND_60d8d6d29e2c4b14f4ea1983baefd36e,Burger King,Burger King,23516,Restaurants and Other Eating Places,Limited-Service Restaurants,722513,40.196205,...,Coimbra,PT,3030-173,"{\n ""Fri"": [\n [\n ""11:00"",\n ""2...","Counter Service,Late Night,Lunch,Fast Food,Dri...",NaT,NaT,2021-08-01,"[\n ""burgerking.pt""\n]",3.513090e+11
2,222-222@62k-hz8-pn5,NaN,SG_BRAND_60d8d6d29e2c4b14f4ea1983baefd36e,Burger King,Burger King,1123,Restaurants and Other Eating Places,Limited-Service Restaurants,722513,41.705784,...,RI,US,2889,"{\n ""Fri"": [\n [\n ""7:00"",\n ""24...","Counter Service,Late Night,Lunch,Fast Food,Dri...",NaT,NaT,2019-07-01,"[\n ""burgerking.com"",\n ""bk.com""\n]",NaN
3,224-222@5z6-vm9-qvf,zzy-222@5z6-vm9-qpv,SG_BRAND_60d8d6d29e2c4b14f4ea1983baefd36e,Burger King,Burger King,9474,Restaurants and Other Eating Places,Limited-Service Restaurants,722513,36.313166,...,CA,US,93245,"{\n ""Fri"": [\n [\n ""6:00"",\n ""22...","Counter Service,Late Night,Lunch,Fast Food,Dri...",NaT,NaT,2019-07-01,"[\n ""bk.com"",\n ""burgerking.com""\n]",NaN
4,222-223@5sb-8jc-4d9,NaN,SG_BRAND_60d8d6d29e2c4b14f4ea1983baefd36e,Burger King,Burger King,22414,Restaurants and Other Eating Places,Limited-Service Restaurants,722513,42.119464,...,IL,US,60110,"{\n ""Fri"": [\n [\n ""6:00"",\n ""22...","Counter Service,Late Night,Lunch,Fast Food,Dri...",NaT,NaT,2019-07-01,"[\n ""bk.com"",\n ""burgerking.com""\n]",1.630912e+10


In [ ]:
image_id_list: list = []

# Iterating over the rows of the df
for index, row in enumerate(safegraph_place.iterrows()):

  # An empty dictionary that will hold our data as we go along
  coordinates_to_image_id_dict: dict = {}

  
  # Converting each row into a dictionary, then extracting the Lat, Long
  latitude, longitude = dict(row[1])['LATITUDE'], dict(row[1])['LONGITUDE'] 

  # Save the longitude and the latitude in the dict defined above
  coordinates_to_image_id_dict['lat'] = latitude
  coordinates_to_image_id_dict['lng'] = longitude


# mly.get_image_close_to
  # Get a `FeatureCollection` given the coordinates defined above
  # define the radius of 35-meter, flat type images and the minum_capture_date and, and convert to a dictionary 
  try:
      data: dict = mly.get_image_close_to(latitude=latitude,
                                            longitude=longitude,
                                            radius=35,
                                            image_type="flat",
                                            min_captured_at="2016-01-01").to_dict()

  except AttributeError:
      pass

  # Define an empty list which will store our image_ids
  coordinates_to_image_id_dict['image_ids'] = []

  # Iterate over the features retrieved
  for index, feature in enumerate(data['features']):


    # Append to the list the image_id from the feature
    coordinates_to_image_id_dict['image_ids'].append(feature['properties']['id'])

  # Finally, append this dictionary to the final result. That is, the image_id_list variable
  image_id_list.append(coordinates_to_image_id_dict)
print (" This is the end of gathering the main")
image_properties: dict = {}

# Useful list of fields we can use, the full list could be seen here: https://www.mapillary.com/developer/api-documentation/
fields_to_pass_for_the_image: list = ['merge_cc',
                                      'mesh',
                                      'altitude',
                                      'sfm_cluster',
                                      'atomic_scale',
                                      'camera_parameters',
                                      'camera_type',
                                      'captured_at',
                                      'compass_angle',
                                      'computed_altitude',
                                      'computed_compass_angle',
                                      'computed_geometry',
                                      'computed_rotation',
                                      'exif_orientation',
                                      'quality_score',
                                      'height',
                                      'width',
                                      'sequence',
                                      'thumb_2048_url']

for index, image_ids in enumerate(image_id_list):

  image_id_list[index]['property_data'] = []

  for image_id in image_ids['image_ids']:
    
    response = Client().get(Entities.get_image(image_id=image_id, fields=fields_to_pass_for_the_image))

    data = json.loads(response.content.decode('utf-8'))

    image_id_list[index]['property_data'].append(data)

# Save all of the data as a JSON
with open('mapillarymetadata.json', 'w') as file_pointer:
  json.dump(image_id_list, file_pointer, indent=4)